In [2]:
#%pip install bayesian-optimization

In [3]:
# implement a dummy Bayesian optimization algorithm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bayes_opt import BayesianOptimization
from bayes_opt import UtilityFunction
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events
from bayes_opt.util import load_logs
from sklearn.gaussian_process.kernels import RBF, Matern # you can try to import other kernels from sklearn as well

### Prepare target flow curve

In [4]:
# Read the CSV file into a DataFrame
df = pd.read_csv('abaqus simulation F-D fitting/Flowcurve_RT.csv')
# print(df)
# Extract the true strain and average true stress columns
trueStrain = df['True strain']
trueStress = df['Avg.True stress']

In [5]:
# Continuous searching space

param_bounds = {
    "c1": (700, 1800),  
    "c2": (0.1 * 1e2, 10 * 1e2) ,    
    "c3": (0.01  * 1e3, 0.1 * 1e3)
}

# Swift Laws

def swift_law(c1,c2,c3,strain):
    c2_temp=c2*1e-14 * 1e-2
    c3_temp=c3* 1e-3
    return c1* np.exp(c3_temp * np.log(c2_temp + strain))

# Note: BO in Bayes-Opt tries to maximize, so you should use the inverse of the loss function.

def lossFunction(**solution):
    #print(solution)
    c1 = solution["c1"]
    c2 = solution["c2"]
    c3 = solution["c3"]
    simStress = swift_law(c1,c2,c3,trueStrain)

    # RMSE loss
    fitness = np.sqrt(np.mean((simStress - trueStress)**2))
    loss = -fitness
    return loss

In [6]:
class BO():
    
    ##################################
    # OPTIMIZER CLASS INITIALIZATION #
    ##################################

    def __init__(self):        
        #############################
        # Optimizer hyperparameters #
        #############################
        
        # maximize parameters
        self.verbose = 1 # 0 for no output, 1 for some output printing
        self.random_state = 123 # random seed
        self.init_points = 100 # number of initial points to sample randomly for Bayesian optimization
        self.iterations = 100 # number of iterations to run Bayesian optimization
        
        # Acquisition function        
        # Low kappa means more exploitation for UCB
        # High kappa means more exploration for UCB
        # Low xi means more exploitation for EI and POI
        # High xi means more exploration for EI and POI
        self.acquisitionFunction = UtilityFunction(kind='ucb', kappa=2.576, xi=0, kappa_decay=1, kappa_decay_delay=0)
        #self.acquisitionFunction = UtilityFunction(kind='poi', kappa=2.576, xi=0, kappa_decay=1, kappa_decay_delay=0)
        #self.acquisitionFunction = UtilityFunction(kind='ei', kappa=2.576, xi=0, kappa_decay=1, kappa_decay_delay=0)
        
        # Gaussian process kernel parameters
        self.GP_kernel = RBF(length_scale=1.0, length_scale_bounds=(1e-3, 1e3)) # RBF kernel
        #self.GP_kernel = Matern(nu=2.5) # Matern kernel
        self.alpha = 1e-6
        self.normalize_y=True
        self.n_restarts_optimizer=5
        
    ##########################
    # OPTIMIZATION FUNCTIONS #
    ##########################

    def initializeOptimizer(self, lossFunction, param_bounds):
        self.param_bounds = param_bounds
        BO_bounds = param_bounds
        bo_instance = BayesianOptimization(
            f = lossFunction,
            pbounds = BO_bounds, 
            verbose = self.verbose,
            random_state = self.random_state,
            bounds_transformer = None,
            allow_duplicate_points = False
        )
        bo_instance.set_gp_params(
            kernel=self.GP_kernel,
            alpha=self.alpha,
            normalize_y=self.normalize_y,
            n_restarts_optimizer=self.n_restarts_optimizer,
            random_state=self.random_state
        )
        self.optimizer = bo_instance

    def run(self):
        self.optimizer.maximize(
            init_points = self.init_points, 
            n_iter = self.iterations,   
            acquisition_function=self.acquisitionFunction
        )
        
    def outputResult(self):
        solution_dict = self.optimizer.max["params"]
        solution_tuple = tuple(solution_dict.items())
        best_solution_loss = self.optimizer.max["target"]
        return solution_dict, solution_tuple, best_solution_loss

In [7]:
BO_instance = BO()
BO_instance.initializeOptimizer(lossFunction, param_bounds)
BO_instance.run()
solution_dict, solution_tuple, best_solution_loss = BO_instance.outputResult()
print(solution_dict)

for param in solution_dict:
    print(f"{param}: {solution_dict[param]}")

print(f"Best solution loss = {best_solution_loss:.4f}")

|   iter    |  target   |    c1     |    c2     |    c3     |
-------------------------------------------------------------
| 2         | -115.8    | 1.306e+03 | 722.3     | 48.08     |
| 5         | -44.57    | 1.182e+03 | 69.08     | 45.82     |
| 7         | -32.3     | 1.285e+03 | 536.5     | 67.1      |
| 138       | -26.75    | 1.448e+03 | 836.7     | 100.0     |
| 145       | -25.6     | 1.429e+03 | 64.27     | 100.0     |
| 177       | -24.91    | 1.435e+03 | 482.0     | 100.0     |
{'c1': 1434.7256845729885, 'c2': 482.03042944631676, 'c3': 100.0}
c1: 1434.7256845729885
c2: 482.03042944631676
c3: 100.0
Best solution loss = -24.9102


In [8]:
# Saving, loading and restarting